In [16]:
# Standard library imports
import math
import os

# Related third party imports
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
from google.cloud import bigquery
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LSTM, LeakyReLU
# from keras.optimizers import Adam
import pydoc_data
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
# Local application/library specific imports
# (None in this example, but this is where they would go)
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense
# Load the environment variables from the .env file
load_dotenv()
google_credentials = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')


In [83]:


PROJECT = "focus-hulling-416322"
DATASET = "2016"
TABLE = "out_and_in_2016_v4"

# Ensure the project, dataset, and table names are correctly formatted
# using backticks to avoid syntax errors
query = f"""
    SELECT *
    FROM `{PROJECT}.{DATASET}.{TABLE}`
"""

client = bigquery.Client(project=PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()


/Users/amermujkanovic/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


Patient_ID    Gender  age_years  region_mod  Relationship_To_Primary_Beneficiary  Patient_Zipcode  Health_Plan_Type  plan_typ  Clinic  drg  Clinic_visits  Myocardial_infarction  chf  pvd  Cardiovascular_d  Respiratory_d  Hypertension  Diabetes_Melitus  Dementia  Kidney_disease  Liver_disease  Diarrheal_disease  Cancer  Metastasis  Connective_tissue_disease  puc  hemiplegia  lymphoma  aids  lohs  Coinsurance  Copay  Deductable   Net           Pay           trauma
2.030200e+04  2       61.0       2           2                                    0.0              0                 6.0       2       0    16             0.0                    0.0  0.0  0.0               0.0            0.0           0.0               0.0       0.0             0.0            0.0                0.0     0.0         0.0                        0.0  0.0         0.0       0.0   0     126.919963   0.0    699.999619   593.379808    1451.939667   0         1
4.011262e+09  1       59.0       4           2               

In [177]:
data['Clinic'].value_counts()

Clinic
2    2476959
1     340194
Name: count, dtype: int64

In [187]:
print(df['clinic_inpatient'].value_counts())
print(df['clinic_outpatient'].value_counts())
print(df['both_clinic'].value_counts())
print(df['gender_female'].value_counts())
print(df['gender_male'].value_counts())

clinic_inpatient
0.0    1789957
1.0     242863
Name: count, dtype: int64
clinic_outpatient
1.0    2032555
0.0        265
Name: count, dtype: int64
both_clinic
0    1845385
1     187435
Name: count, dtype: int64
gender_female
0.0    1262051
1.0     770769
Name: count, dtype: int64
gender_male
1.0    1262051
0.0     770769
Name: count, dtype: int64


In [185]:
# Cleaning the column names
df = data.clean_names()

# Changin the values inside the clinic column for onehot encoding
value_map = {1: 'Inpatient', 2: 'Outpatient'}
df['clinic'] = df['clinic'].map(value_map)

#Chaning the values for region_mod
value_map_2 = {'1': 'northeast', '2':'northcentral', '3':'south', '4':'west', '5':'unknown'}
df['region_mod'] = df['region_mod'].map(value_map_2)

#chaning the gender varible
value_map_3 = {'1':'male', '2':'female'}
df['gender'] = df['gender'].map(value_map_3)

# # Creating values for Plantype
# value_map_4 = {1: 'basic', 2: 'comprihensive', 3:'epo', 4:'hmo', 5:'pos', 6:'ppo', 7:'pos2', 8:'cdhp', 9:'hdhp'}


def encode_and_bind(original_dataframe, features_to_encode):
    from sklearn.preprocessing import OneHotEncoder

    # Convert a single feature name to a list
    if isinstance(features_to_encode, str):
        features_to_encode = [features_to_encode]

    # Check if the features exist in the dataframe
    for feature in features_to_encode:
        if feature not in original_dataframe.columns:
            raise ValueError(f"Feature '{feature}' not found in the dataframe")

    # One-hot encoding
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(original_dataframe[features_to_encode])
    encoded_features = encoder.transform(original_dataframe[features_to_encode])

    # Add new encoded columns to the dataframe
    original_dataframe[encoder.get_feature_names_out(features_to_encode)] = encoded_features

    # Drop the original columns
    original_dataframe.drop(columns=features_to_encode, inplace=True)

    # Return the modified dataframe
    return original_dataframe

features_to_encode = ['clinic', 'region_mod', 'gender']

encode_and_bind(df, features_to_encode)

value_to_encode = [183, 184, 185, 521, 522, 533, 534, 545, 536, 542, 543, 544, 562, 563]
df['trauma'] = df['drg'].apply(lambda x: 1 if x in value_to_encode else 0)



#cleaning names again
df = df.clean_names()

group_columns = ['patient_id', 'gender_male', 'relationship_to_primary_beneficiary']

agg_dict = {col: 'sum' for col in df.columns if col not in group_columns}
agg_dict['age_years'] = 'mean'
df = df.groupby(group_columns).agg(agg_dict).reset_index()



#replacing the values greater than 1


# #sorting out patient_id column
df['patient_id'] = df['patient_id'].astype(str)
df['patient_id'] = df['patient_id'].str.slice(0, -2)

# #eliminating the negative pay values

df = df[(df['age_years'] > 17) & (df['pay'].between(2,1000000))]



#removing the duplicates
duplicates = df.duplicated(subset='patient_id', keep=False)
duplicate_rows= df[duplicates]
duplicate_rows.sort_values(by='patient_id')
df.drop_duplicates(inplace=True)
df['both_clinic'] = np.where((df['clinic_inpatient'] == 1.0) & (df['clinic_outpatient'] == 1.0), 1,0)

# CCI score
df['cci'] = 0
df['cci']= pd.cut(df['age_years'],
                    bins=[0, 49, 59, 69, 79, float('inf')],
                    labels=[0, 1, 2, 3, 4],
                    right=False).astype(int)
df.loc[df['myocardial_infarction'] == 1, 'cci'] +=1
df.loc[df['chf'] == 1, 'cci'] +=1
df.loc[df['pvd'] == 1, 'cci'] +=1
df.loc[df['respiratory_d'] == 1, 'cci'] +=1
df.loc[df['connective_tissue_disease'] == 1, 'cci'] +=1
df.loc[df['liver_disease'] == 1, 'cci'] +=1
df.loc[df['diabetes_melitus'] == 1, 'cci'] +=1
df.loc[df['kidney_disease'] == 1, 'cci'] +=1
df.loc[df['puc'] == 1, 'cci'] +=1
df.loc[df['hemiplegia'] == 1, 'cci'] +=2
df.loc[df['lymphoma'] == 1, 'cci'] +=2
df.loc[df['aids'] == 1, 'cci'] +=6
df.loc[df['cancer'] == 1, 'cci'] +=2
df.loc[df['metastasis'] == 1, 'cci'] +=6

scaling_columns = ['lohs', 'clinic_visits', 'age_years']
def scaler_minmax(data_frame, scaling_columns):
    minmax = MinMaxScaler()
    if isinstance(scaling_columns, str):  # If a single column name is passed as a string
        scaling_columns = [scaling_columns]  # Convert it to a list

    minmax.fit(data_frame[scaling_columns])
    data_frame[scaling_columns] = minmax.transform(data_frame[scaling_columns])

scaler_minmax(df, scaling_columns)

columns_to_replace = [
    'myocardial_infarction', 'trauma', 'chf', 'pvd', 'cardiovascular_d',
    'respiratory_d', 'hypertension', 'diabetes_melitus', 'dementia',
    'kidney_disease', 'liver_disease', 'diarrheal_disease', 'cancer',
    'metastasis', 'puc', 'hemiplegia', 'lymphoma', 'aids',
    'connective_tissue_disease', 'region_mod_northcentral',
    'region_mod_northeast', 'region_mod_south', 'region_mod_west',
    'region_mod_unknown', 'clinic_outpatient', 'clinic_inpatient', 'both_clinic', 'gender_female', 'gender_male'
]

new_value = 1

for column in columns_to_replace:
    df[column] = df[column].apply(lambda x: new_value if 2.0 <= x <= 200.0 else x)


/Users/amermujkanovic/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [199]:
print(df['clinic_inpatient'].value_counts())
print(df['clinic_outpatient'].value_counts())
print(df['both_clinic'].value_counts())
print(df['gender_female'].value_counts())
print(df['gender_male'].value_counts())
print(df['region_mod_unknown'].value_counts())
print(df['age_years'].mean())
print(df['diabetes_melitus'].value_counts())
print(df['lohs'].mean())


clinic_inpatient
0.0    1789957
1.0     242863
Name: count, dtype: int64
clinic_outpatient
1.0    2032555
0.0        265
Name: count, dtype: int64
both_clinic
0    1845385
1     187435
Name: count, dtype: int64
gender_female
0.0    1262051
1.0     770769
Name: count, dtype: int64
gender_male
1.0    1262051
0.0     770769
Name: count, dtype: int64
region_mod_unknown
0.0    2024214
1.0       8606
Name: count, dtype: int64
0.6674917267219399
diabetes_melitus
0.0    1715965
1.0     316855
Name: count, dtype: int64
0.0019333601453801641


In [146]:

df.columns

Index(['patient_id', 'gender_male', 'relationship_to_primary_beneficiary',
       'age_years', 'patient_zipcode', 'health_plan_type', 'plan_typ', 'drg',
       'clinic_visits', 'myocardial_infarction', 'chf', 'pvd',
       'cardiovascular_d', 'respiratory_d', 'hypertension', 'diabetes_melitus',
       'dementia', 'kidney_disease', 'liver_disease', 'diarrheal_disease',
       'cancer', 'metastasis', 'connective_tissue_disease', 'puc',
       'hemiplegia', 'lymphoma', 'aids', 'lohs', 'coinsurance', 'copay',
       'deductable', 'net', 'pay', 'trauma', 'clinic_inpatient',
       'clinic_outpatient', 'region_mod_northcentral', 'region_mod_northeast',
       'region_mod_south', 'region_mod_unknown', 'region_mod_west',
       'gender_female', 'both_clinic', 'cci'],
      dtype='object')

In [168]:
import matplotlib.pyplot as plt
# plt.hist(df['cci'], bins=30, edgecolor='black')
df['cci'].describe()

count    2.032820e+06
mean     1.201522e+00
std      1.207555e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      1.500000e+01
Name: cci, dtype: float64

### outliers ###

## Preparing the X and the y

In [196]:
# Preparing the data

X = df.drop(columns=['pay', 'clinic_outpatient', 'patient_zipcode', 'patient_id', 'health_plan_type', 'plan_typ', 'drg', 'relationship_to_primary_beneficiary', 'coinsurance', 'copay', 'deductable', 'net', 'region_mod_unknown'])

y = df['pay']


In [190]:
X

,gender_male,age_years,clinic_visits,myocardial_infarction,chf,pvd,cardiovascular_d,respiratory_d,hypertension,diabetes_melitus,...,trauma,clinic_inpatient,clinic_outpatient,region_mod_northcentral,region_mod_northeast,region_mod_south,region_mod_west,gender_female,both_clinic,cci
0,0.0,0.914894,0.014272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0,2
1,1.0,0.978723,0.001903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,2
2,0.0,0.744681,0.074215,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1,2
3,1.0,0.914894,0.059943,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1,2
4,1.0,0.893617,0.050428,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298456,0.0,0.872340,0.006660,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0,3
2298457,1.0,0.297872,0.006660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0
2298458,1.0,0.723404,0.028544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1
2298459,1.0,0.382979,0.003806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0


## The Decision Tree Regressor

In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cv_result = cross_validate(DecisionTreeRegressor(max_depth=8, min_samples_leaf=7, min_samples_split=30), X_train, y_train, cv=5, scoring='r2')
test_score = cv_result['test_score'].mean()
print(test_score)


0.6086124662781021


## The Random Forrest Regressor

## Spliting the dataset

In [138]:
# Random forrest regressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## The Grid Search for Hyperparametrs

## The Model

In [139]:
cv_result = cross_validate(RandomForestRegressor(max_depth=8,
                                                 min_samples_leaf=7,
                                                 min_samples_split=30,
                                                 n_estimators=200,
                                                 criterion='squared_error',
                                                 max_features='sqrt'),
                           X_train, y_train, cv=5, scoring='r2')


test_score_rf = cv_result['test_score'].mean()

In [110]:
rf = RandomForestRegressor(n_estimators=200, min_samples_leaf=7, min_samples_split=30, max_depth=8)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, min_samples_leaf=7, min_samples_split=300,
                      n_estimators=200)

## Predictions

In [111]:
y_pred_rf = rf.predict(X_test)
errors = abs(y_pred_rf - y_test)
print('Mean absolute error:', round(np.mean(errors),2), 'dollars')

Mean absolute error: 8596.17 dollars


# Gradient Boosting Machine

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [120]:
gbm = cross_validate(GradientBoostingRegressor(n_estimators=200,
                                               learning_rate=0.1,
                                               min_samples_leaf=7,
                                               max_depth=8,
                                               min_samples_split=30,
                                               random_state=42),
                     X_train,
                     y_train,
                     cv=5,
                     scoring='r2')

test_score_gbm = gbm['test_score'].mean()
test_score_gbm

In [122]:
gbm = GradientBoostingRegressor(n_estimators=200,
                                min_samples_split=30,
                                min_samples_leaf=7,
                                max_depth=8 ,
                                learning_rate=0.1,
                                random_state=42)
gbm.fit(X_train, y_train)

y_pred = gbm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
#670160468.497997

Mean Squared Error: 662402098.5054079


# Comparing models

In [143]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import regularizers
# Load dataset

X = X.astype(float)

# # Standardize features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalizing the dataset
# from sklearn.preprocessing import normalize
# X_train_normalized = normalize(X_train, axis=0)
# X_test_normalized = normalize(X_test, axis=0)

# # Architecture one with relu and dropout rate
model = models.Sequential()
model.add(layers.Dense(30, activation= 'relu', input_shape=(X.shape[1],),
                       kernel_regularizer=regularizers.l1(0.01)))
model.add(layers.Dense(30, activation='relu',
                       kernel_regularizer=regularizers.l1(0.01)))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='relu',
                       kernel_regularizer=regularizers.l1(0.01)))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))


# eraly stopping
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=15,
                               verbose =1,
                               restore_best_weights=True)


# # Compile the model
model.compile(optimizer=Adam(learning_rate = 0.01),
              loss='mean_absolute_error',
              metrics=[('mae', 'accuracy')])

# Train the model
model.fit(X_train, y_train, epochs=59, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

# Evaluate the model on test data
results = model.evaluate(X_test, y_test)
# print(f'Test loss: {test_loss}')

Epoch 1/59
35575/35575 [==============================] - 35s 988us/step - loss: 15598.3105 - mae: 15593.2246 - accuracy: 0.0000e+00 - val_loss: 15163.8262 - val_mae: 15163.7939 - val_accuracy: 0.0000e+00
Epoch 2/59
35575/35575 [==============================] - 37s 1ms/step - loss: 92259.7422 - mae: 92256.9688 - accuracy: 0.0000e+00 - val_loss: 14915.6836 - val_mae: 14915.1221 - val_accuracy: 0.0000e+00
Epoch 3/59
 6427/35575 [====>.........................] - ETA: 22s - loss: 14807.4258 - mae: 14806.4443 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [61]:
results

# 7788.56640625, 7788.56640625


[7868.2763671875, 7836.2109375, 0.0]

In [187]:
np.sqrt(results[0])
# 32131.827461257162

26110.428874302313